We are handling two kinds of main dataset:
- the EPS 
- stock's return

In [4]:
import pandas as pd
import numpy as np
import os
import dask.dataframe as dd
import warnings
warnings.filterwarnings('ignore')

### EPS

In [5]:
eps_raw = pd.read_csv('eps.csv')
print(eps_raw.columns)
print(eps_raw.info())

Index(['id', 'name', 'date', 'type', 'eps'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290989 entries, 0 to 290988
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      290989 non-null  int64  
 1   name    290989 non-null  object 
 2   date    290989 non-null  object 
 3   type    290989 non-null  object 
 4   eps     290966 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 11.1+ MB
None


In [6]:
# keep only the consolidated statements
eps_raw = eps_raw[eps_raw['type'] == 'A'].copy()
eps_raw['date'] = pd.to_datetime(eps_raw['date'], format='%Y-%m-%d')
# count every 06/30 or 12/31
eps_raw = eps_raw[
    ((eps_raw['date'].dt.month == 6) & (eps_raw['date'].dt.day == 30)) |
    ((eps_raw['date'].dt.month == 12) & (eps_raw['date'].dt.day == 31))
].copy()
eps_raw = eps_raw.drop('type', axis=1)
eps_raw['Year'] = eps_raw['date'].dt.year
# calculate the full year eps
eps_full_year = eps_raw[eps_raw['date'].dt.month == 12].copy()
eps_full_year = eps_full_year[['id', 'Year', 'eps']].rename(columns={'eps': 'EPS_FY'})
eps_raw = pd.merge(eps_raw, eps_full_year, on=['id', 'Year'], how='left')
# sort id and date
eps_raw = eps_raw.sort_values(by=['id', 'date']).reset_index(drop=True)
eps_raw['EPS_diff'] = eps_raw.groupby(['id', 'Year'])['eps'].diff()

We define $$\text{EPS\_diff} = \text{EPS\_FY} - \text{EPS\_HY}$$

In [7]:
eps_raw['EPS_diff'] = eps_raw.groupby(['id', 'Year'])['eps'].diff()

In [8]:
is_month_6 = eps_raw['date'].dt.month == 6
eps_raw['eps'] = np.where(
    is_month_6,
    eps_raw['EPS_FY'],
    eps_raw['EPS_FY'] + eps_raw['EPS_diff']
)
eps_raw = eps_raw.drop(['Year', 'EPS_FY', 'EPS_diff'], axis=1)

We are slicing the data by some financial indicators:

$$\text{UE}_{i, t} = \text{EPS}_{i, t} - \text{EPS}_{i, t-2} $$

where:
- $\text{UE}_{i, t}$: unexpected returns of stock $i$ at time $t$
- $\text{EPS}_{i, t}$: earning per share of stock $i$ at time $t$
- $\text{EPS}_{i, t-2}$: earning per share of stock $i$ at time $t-2$

Dividing this indicator by its rolling standard deviation to obtain:

$$\text{SUE}_{i, t}=\frac{\text{UE}_{i, t}}{\sigma_{i, t}}$$

In [9]:
eps_raw['UE'] = eps_raw.groupby('id')['eps'].transform(lambda x: x.diff(periods=2))
eps_raw['std'] = eps_raw.groupby('id')['UE'].transform(lambda x: x.rolling(4).std())
eps_raw['SUE'] = eps_raw['UE'] / eps_raw['std']
eps_raw.dropna(subset=['SUE'], inplace=True)
# split SUE into 10 groups
eps_raw['SUE decile'] = eps_raw.groupby('date')['SUE'].transform(
    lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1
)

Then add in the announcement date

In [10]:
ann_raw = pd.read_csv('ann.csv')
ann_raw['date'] = pd.to_datetime(ann_raw['date'], format='%Y-%m-%d')
ann_raw['announce'] = pd.to_datetime(ann_raw['announce'], format='%Y-%m-%d')
ann_key = ann_raw[['id', 'date', 'announce']].set_index(['id', 'date'])
eps_raw_indexed = eps_raw.set_index(['id', 'date'])
eps_final = eps_raw_indexed.join(ann_key, how='left')
eps_final = eps_final.reset_index()
# delete *ST
stkid2delete = eps_final['name'].str.contains(r'^(ST|\*ST)', regex=True, na=False)
eps_final = eps_final[~stkid2delete]

### Stock Returns

In [11]:
core_count = os.cpu_count()
print(core_count)

18


In [12]:
def process_stock_data(df):
    stock_in_main = df[~df['Markettype'].isin([2, 8, 16, 32])]
    return stock_in_main.sort_values(by=['Trddt', 'Stkcd'])

In [13]:
file_paths = [f'{i}.csv' for i in range(1, 9)]
ddf = dd.read_csv(file_paths, blocksize=None)
ddf_ = ddf.map_partitions(process_stock_data, meta=ddf._meta)
stock_merge = ddf_.compute(num_workers=os.cpu_count())

Merge market returns

In [18]:
mkt_df = pd.read_csv('mkt_return.csv').query('Markettype == 5').drop(columns=['Markettype'])
stock_df = stock_merge.merge(mkt_df, on='Trddt', how='left')
if 'Markettype' in stock_df.columns:
    stock_df = stock_df.drop(columns=['Markettype'])
stock_df['ARs'] = stock_df['Dretwd'] - stock_df['Cdretwdeq']

Save data

In [20]:
eps_final.to_parquet('processed_eps.parquet')
stock_df.to_parquet('processed_stock.parquet')